In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# --- AYARLAR ---
# Veri setinin bilgisayardaki konumu
VERI_YOLU = r"/Users/hamzahakverir/Downloads/PKLot/PKLotSegmented/PUC/Sunny/2012-09-12"
MODEL_KAYIT_ADI = "Hamza_Hakverir_Dropout_Model.pth"
EPOCH_SAYISI = 3
# --------------

# 1. VERİ HAZIRLIĞI
# Resimleri 64x64 piksel boyutuna getirip tensor formatina ceviriyoruz
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

print("-" * 60)
print("CNN MODEL TRAINING PROCESS (EĞİTİM SÜRECİ)")
print("-" * 60)
print(f"[INFO] Veri Yolu / Data Path: .../{VERI_YOLU.split('/')[-1]}")

try:
    # Klasor yapisindan verileri okuyoruz (ImageFolder)
    tum_veri = datasets.ImageFolder(root=VERI_YOLU, transform=transform)
    print(f"[INFO] Toplam Görsel (Total Images): {len(tum_veri)}")
    
    # Veriyi %80 Egitim, %20 Test olacak sekilde ayirdim
    egitim_boyutu = int(0.8 * len(tum_veri))
    test_boyutu = len(tum_veri) - egitim_boyutu
    egitim_seti, test_seti = random_split(tum_veri, [egitim_boyutu, test_boyutu])
    
    train_loader = DataLoader(egitim_seti, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_seti, batch_size=32, shuffle=False)
    
    print(f"[STATUS] Veri Seti Ayrıştırıldı:")
    print(f"   > Training Set (Eğitim) : {len(egitim_seti)} adet")
    print(f"   > Test Set (Test)       : {len(test_seti)} adet")
    print("-" * 60)

except Exception as e:
    print(f"[ERROR] Veri yükleme hatası: {e}")
    train_loader = None

# 2. MODEL MİMARİSİ (CNN + Dropout)
class DropoutCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 2 adet konvolüsyon katmani ve max pooling
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        # Overfitting'i (ezberlemeyi) engellemek icin %50 Dropout ekledim
        self.dropout = nn.Dropout(0.5) 
        
        # Siniflandirma katmani (Dolu/Bos)
        self.fc1 = nn.Linear(32 * 16 * 16, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Katmanlar arasi gecisler
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16) 
        x = self.dropout(x)          
        x = self.fc1(x)
        return x

# 3. EĞİTİM 
if train_loader is not None:
    model = DropoutCNN()
    # Hata hesaplama ve optimizasyon ayarlari
    kayip_fonksiyonu = nn.CrossEntropyLoss()
    optimizasyon = optim.Adam(model.parameters(), lr=0.001)

    print(f"Model Eğitimi Başlıyor / Training Started ({EPOCH_SAYISI} Epochs)...")
    model.train()
    
    # Egitim dongusu
    for epoch in range(EPOCH_SAYISI): 
        dogru_tahmin = 0
        toplam_veri = 0
        kosu_kaybi = 0.0 
        
        for data in train_loader:
            resimler, etiketler = data
            
            
            optimizasyon.zero_grad()
            tahminler = model(resimler)
            
            
            loss = kayip_fonksiyonu(tahminler, etiketler)
            loss.backward()
            optimizasyon.step()
            
            kosu_kaybi += loss.item()
            _, predicted = torch.max(tahminler.data, 1)
            toplam_veri += etiketler.size(0)
            dogru_tahmin += (predicted == etiketler).sum().item()
        
        epoch_loss = kosu_kaybi / len(train_loader)
        epoch_acc = 100 * dogru_tahmin / toplam_veri
        
        print(f">> Epoch {epoch+1}/{EPOCH_SAYISI} | Loss (Hata): {epoch_loss:.4f} | Accuracy (Başarı): %{epoch_acc:.2f}")
    
    print("-" * 60)
    
    # TEST AŞAMASI
    print("[INFO] Test İşlemi Başlatılıyor...")
    model.eval() 
    dogru_tahmin = 0
    toplam_veri = 0
    
    with torch.no_grad():
        for data in test_loader:
            resimler, etiketler = data
            tahminler = model(resimler)
            _, predicted = torch.max(tahminler.data, 1)
            toplam_veri += etiketler.size(0)
            dogru_tahmin += (predicted == etiketler).sum().item()
    
    final_acc = 100 * dogru_tahmin / toplam_veri
    print(f"🏆 FINAL RESULT (Test Seti): %{final_acc:.2f}")
    print("-" * 60)
    
    # Modeli dosyaya kaydet
    torch.save(model.state_dict(), MODEL_KAYIT_ADI)
    print(f"[INFO] Model Dosyası Kaydedildi: {MODEL_KAYIT_ADI}")

------------------------------------------------------------
CNN MODEL TRAINING PROCESS (EĞİTİM SÜRECİ)
------------------------------------------------------------
[INFO] Veri Yolu / Data Path: .../2012-09-12
[INFO] Toplam Görsel (Total Images): 8999
[STATUS] Veri Seti Ayrıştırıldı:
   > Training Set (Eğitim) : 7199 adet
   > Test Set (Test)       : 1800 adet
------------------------------------------------------------
Model Eğitimi Başlıyor / Training Started (3 Epochs)...
>> Epoch 1/3 | Loss (Hata): 0.1135 | Accuracy (Başarı): %95.64
>> Epoch 2/3 | Loss (Hata): 0.0209 | Accuracy (Başarı): %99.53
>> Epoch 3/3 | Loss (Hata): 0.0113 | Accuracy (Başarı): %99.62
------------------------------------------------------------
[INFO] Test İşlemi Başlatılıyor...
🏆 FINAL RESULT (Test Seti): %96.61
------------------------------------------------------------
[INFO] Model Dosyası Kaydedildi: Hamza_Hakverir_Dropout_Model.pth
